# FedCAD: Centralized vs Federated (orchestration)

This notebook launches centralized and federated experiments via existing entry points:
- Centralized: `python train_central.py`
- Federated: `flwr run .`

Metrics tracked in W&B:
- Centralized: `train_loss`, `test_loss`, `test_acc` (already logged by `train_central.py`)
- Federated: depends on your Flower app (ensure it logs to the same W&B project/group)

Recommended seeds for a seminar: 3 (e.g., `[0, 1, 2]`). Use 5 if time permits for tighter confidence intervals.

In [ ]:
import os, sys, subprocess, shutil, time
import torch

# W&B configuration
def load_wandb_config():
    """Load WandB credentials from config file (optional for reproducibility)"""
    config = {}
    config_path = './.wandb_config'
    try:
        with open(config_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    config[key] = value
        return config.get('WANDB_ENTITY'), config.get('WANDB_PROJECT')
    except FileNotFoundError:
        return None, None

entity, project_name = load_wandb_config()
WANDB_MODE = "online"               # set to "offline" or "disabled" if needed

SEEDS = list(range(0, 66)) 

# Centralized hyperparameters
EPOCHS_CENTRAL = 30
LR_CENTRAL = 1e-3
EVAL_INTERVAL = 1

# Device
USE_GPU_FLAG = torch.cuda.is_available()
print("Device:", "cuda" if USE_GPU_FLAG else "cpu")
print("Seeds:", SEEDS)

def run_cmd(cmd_list, extra_env=None, cwd=None):
    env = os.environ.copy()
    if extra_env:
        env.update({k: str(v) for k, v in extra_env.items()})
    print("\n>>>", " ".join(cmd_list))
    print("    in:", cwd or os.getcwd())
    print("    with env overrides:", extra_env or {})
    proc = subprocess.Popen(cmd_list, cwd=cwd, env=env)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {proc.returncode}: {' '.join(cmd_list)}")

## Centralized experiments

Runs `train_central.py` once per seed. The script already logs: `train_loss`, `test_loss`, `test_acc`.

Note: `train_central.py` does not expose a `--seed` flag. If you need strict reproducibility per seed, add seeding to the script; otherwise each run will use random initialization.

In [ ]:
for seed in SEEDS:
    # W&B grouping so runs are easy to compare
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "centralized",
        "WANDB_TAGS": "centralized,notebook",
        "WANDB_ENTITY": entity,

        # Optional: many apps read SEED from env; train_central.py currently does not
        "SEED": seed,
    }

    cmd = [sys.executable, "train_central.py",
           "--epochs", str(EPOCHS_CENTRAL),
           "--lr", str(LR_CENTRAL),
           "--eval-interval", str(EVAL_INTERVAL),
           "--wandb-mode", WANDB_MODE]
    if USE_GPU_FLAG:
        cmd.append("--gpu")

    print(f"\n=== Centralized run, seed={seed} ===")
    run_cmd(cmd, extra_env=env_overrides, cwd=os.getcwd())
    time.sleep(1)  # small gap between runs

In [ ]:
import os

# Check current state
print("1. Current directory:", os.getcwd())
print("2. pyproject.toml exists:", os.path.exists("pyproject.toml"))
print("3. Full path:", os.path.abspath("pyproject.toml"))

# List files to confirm
print("\n4. Files in current directory:")
for f in os.listdir("."):
    print(f"   {f}")

## Federated experiments (Flower Simulation Engine)

Runs `flwr run .` once per seed. Ensure your Flower app logs to W&B (same project) with metrics comparable to centralized (e.g., `global_train_loss`, `global_test_loss`, `global_test_acc`).

Tip: If your Flower app reads `WANDB_*` env vars and `SEED`, the grouping and seeds below will apply.

In [ ]:
def run_flower(extra_env=None):
    # Prefer CLI if available; fallback to `python -m flwr`
    if shutil.which("flwr"):
        run_cmd(["flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())
    else:
        run_cmd([sys.executable, "-m", "flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())

entity, project_name = load_wandb_config()

for seed in SEEDS:
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "federated",
        "WANDB_TAGS": "federated,notebook",
        "WANDB_ENTITY": entity,
        "SEED": seed,
    }
    print(f"\n=== Federated run (Flower), seed={seed} ===")
    run_flower(extra_env=env_overrides)
    time.sleep(1)